In [8]:
from collections import Counter
from datetime import datetime
import json
import pickle
import numpy as np
from matplotlib import pyplot as plt
# caleb_path = '/Users/CalebKaijiLu/Documents/yelp_dataset/'
caleb_path = '/home/caleb/schoolwork/yelp_dataset/'
selva_path = ''

# Single Review Gender Prediction

In [9]:
with open("tokenizer_gender_single_server.pickle", "rb") as f:
       tokenizer = pickle.load(f)

model = load_model("yelp_gender_single_model_server.hdf5")

In [16]:
# replace with the data you want to classify
newtexts = ["My wife took me to dinner",'My husband took me to dinner','My friend took me to dinner','I went with my kids']
# note that we shouldn't call "fit" on the tokenizer again
sequences = tokenizer.texts_to_sequences(newtexts)
data = pad_sequences(sequences, maxlen=300)

# get predictions for each of your new texts
predictions = model.predict(data)
print(predictions)


[[ 0.02197664]
 [ 0.91792458]
 [ 0.540914  ]
 [ 0.60598183]]


In [40]:
# replace with the data you want to classify
newtexts = ['the food is amazing','Oh my god the food is amazing','oh my god the food is sooooo amazing','oh my god the food is sooooo amazing! I love this place!!','oh my god the food is sooooo amazing!! I luv this place!! ']
# note that we shouldn't call "fit" on the tokenizer again
sequences = tokenizer.texts_to_sequences(newtexts)
data = pad_sequences(sequences, maxlen=300)

# get predictions for each of your new texts
predictions = model.predict(data)
print(predictions)


[[ 0.49949434]
 [ 0.56273288]
 [ 0.75836354]
 [ 0.84929174]
 [ 0.87533987]]


# User-based Review Gender Prediction (TO DO)

In [ ]:
with open("tokenizer_gender_single_server.pickle", "rb") as f:
       tokenizer = pickle.load(f)

model = load_model("yelp_sentiment_model_server.hdf5")

In [ ]:
# replace with the data you want to classify
newtexts = ["Oh My God the Food Is So Bad"]
word_sequences = text_to_word_sequence(newtexts[0])
# note that we shouldn't call "fit" on the tokenizer again
sequences = tokenizer.texts_to_sequences(newtexts)
data = pad_sequences(sequences, maxlen=300)

# get predictions for each of your new texts
predictions = model.predict(data)
print(predictions)
print(word_sequences)

# Explanations